<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/gp_new_model_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* BERT

https://github.com/SKTBrain/KoBERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


* data 불러오기

In [ ]:
import csv

with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/train_data_1-3.csv', 'r') as data:
  train_data= csv.reader(data, delimiter=',')

with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data.csv', 'r') as data:
  test_data= csv.reader(data, delimiter=',')

In [ ]:
import pandas as pd

train_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/train_data_1-3.csv') # encoding='cp949'
test_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data.csv')

In [ ]:
train_df= train_df[['title', 'text', 'emotion', 'emotion & summarize']]
test_df= test_df[['title', 'text', 'emotion']]
# train_df.head()

In [ ]:
train_df['emotion'].value_counts()

기쁨         241
슬픔         221
unknown     59
두려움         39
분노          38
Name: emotion, dtype: int64

In [ ]:
test_df['emotion'].value_counts()

기쁨         15
슬픔          5
unknown     5
분노          4
두려움         2
Name: emotion, dtype: int64

In [ ]:
train_df['train_text']=   "["+ train_df['title'].map(str) + "] " + train_df['text']
train_df= train_df[['train_text', 'emotion']]

test_df['test_text']=  "["+ test_df['title'].map(str) + "] " + test_df['text']
test_df= test_df[['test_text', 'emotion']]
# train_df.head()

In [ ]:
# 기쁨:0 분노:1 혐오:2 두려움:3 슬픔:4 unknown:5
train_df= train_df.replace('기쁨', 0)
train_df= train_df.replace('분노', 1)
train_df= train_df.replace('두려움', 2)
train_df= train_df.replace('슬픔', 3)
train_df= train_df.replace('unknown', 4)

train_df.head()

,train_text,emotion
0,[걱정] 오늘도 일기를 쓰기 위해 연필을 들었다. 오늘은 2월 5일. 벌써 202...,2.0
1,[20141008] 오늘 아침도 여전히 졸렸다. 잠을 적게 자는 것도 아니고 피곤...,1.0
2,[나의 요즘] 사람과 사람이 만났을 때 가장 많이 하는 말은 '요즘 뭐하고 지내?...,2.0
3,[편입일기.. ] 평생을 열심히 살아본 적이 없다고 스스로 생각했다. 성공한 인생...,3.0
4,[백수의 하루] 전날 난 일찍자고 일찍 일어나서 일반인의 하루를 보내리라 결심을 ...,2.0


In [ ]:
# 기쁨:0 분노:1 혐오:2 두려움:3 슬픔:4 unknown:5
test_df= test_df.replace('기쁨', 0)
test_df= test_df.replace('분노', 1)
test_df= test_df.replace('두려움', 2)
test_df= test_df.replace('슬픔', 3)
test_df= test_df.replace('unknown', 4)

test_df.head()

,test_text,emotion
0,[2018년 3월 29일 목요일] 나를 사랑하는 연습 어떻게 하면 나를 사랑할 수...,3
1,[김동률 오래된콘서트를 기념하며] 김동률이 부르는 노래와 나의 인연은 2015년 ...,0
2,[성인간호학 중환자실 실습후기] 2017 1월 2일 월요일 수간호사선선생님의 ot...,4
3,[반포한강에서] 바람이 꽤 쌀쌀해 콧물을 훌쩍이는 날씨에 반포한강에 왔다. 고속버...,4
4,[햄버거를 먹었다.] 말 그대로 햄버거를 먹었다. 아버지와 둘이서. 원래 아버지는...,0


In [ ]:
# 결측값 제거 (단, 중복은 제거하지 않음)
def drop_na(df):
  df= df.dropna()
  df= df.reset_index(drop=True)
  return df

In [ ]:
train_df= drop_na(train_df)
test_data= drop_na(test_df)

In [ ]:
train_df['emotion']= train_df['emotion'].astype(int)
test_df['emotion']= test_df['emotion'].astype(int)

# train_df.head()

In [ ]:
# gpt2 max length = 1024
# 본문은 800자로 제한 (gpt에 들어가는 원문 크기를 따름)
# 본문은 앞 400자, 뒤 400자를 잘라서 합침

In [ ]:
max(train_df['train_text'].apply(len))

9302

In [ ]:
i=0
for text in train_df['train_text']:
  if len(text) > 800:
    train_df['train_text'][i] = train_df['train_text'][i][0:400] + train_df['train_text'][i][len(text)-400:len(text)]
  i += 1

i=0
for text in test_df['test_text']:
  if len(text) > 800:
    test_df['test_text'][i] = test_df['test_text'][i][0:400] + test_df['test_text'][i][len(text)-400:len(text)]
  i += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
max(train_df['train_text'].apply(len))

800

In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(bert)_train_data.tsv', sep='\t')
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(bert)_test_data.tsv', sep='\t')

* BERT: classification

https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

https://tech-diary.tistory.com/31

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9clcrld5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9clcrld5


In [ ]:
# !git clone https://github.com/SKTBrain/KoBERT.git

In [ ]:
%cd KoBERT

/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERT


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERT
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15708 sha256=01fa490f4a83527194e1af1ce79d01bba473580b233c0d6104b3b53a6dcfc9cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-8tl28k3r/wheels/b9/13/65/86f3247f6209d6a89816eb9fbe559035709b69e1190b67b9cf
Successfully built kobert
  Attempting uninstall: kobert
    Found existing installation: kobert 0.2.3
    Uninstalling kobert-0.2.3:
      Successfully 

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yg1ql8w9
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yg1ql8w9


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
device = torch.device("cpu")

## GPU
# device = torch.device("cuda")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# 1lvjJJaAhs-xg3uyEoRva-a03QF-l_jgf
# 1AkyffYxqXPBQSNbaWD6zkeF9c0SIWfXE

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lvjJJaAhs-xg3uyEoRva-a03QF-l_jgf' -O datatrain.tsv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1AkyffYxqXPBQSNbaWD6zkeF9c0SIWfXE' -O datatest.tsv


--2022-08-18 21:02:03--  https://docs.google.com/uc?export=download&id=1lvjJJaAhs-xg3uyEoRva-a03QF-l_jgf
Resolving docs.google.com (docs.google.com)... 173.194.215.101, 173.194.215.138, 173.194.215.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4s1c1o0gb4701ocgvcjfuhn1me4uchib/1660856475000/06373414866148795583/*/1lvjJJaAhs-xg3uyEoRva-a03QF-l_jgf?e=download&uuid=643559ea-babd-4400-b96c-5fe02eb01d9a [following]
--2022-08-18 21:02:04--  https://doc-0o-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4s1c1o0gb4701ocgvcjfuhn1me4uchib/1660856475000/06373414866148795583/*/1lvjJJaAhs-xg3uyEoRva-a03QF-l_jgf?e=download&uuid=643559ea-babd-4400-b96c-5fe02eb01d9a
Resolving doc-0o-3g-docs.googleusercontent.com (doc-0o-3g-docs.googleusercontent.com)... 172.217.193.132, 2607:

In [ ]:
dataset_train = nlp.data.TSVDataset("datatrain.tsv", field_indices=[1, 2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("datatest.tsv", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 512
batch_size = 4
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# dataloader 확인

# for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
#   print('token_ids:' , token_ids)
#   print('valid_lenght: ', valid_length)
#   print( 'segment_ids:', segment_ids)
#   print('labels: ',  label)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# batch_size = 4
# num_epochs = 30 # gpu로 하려면 수동으로 1* 30회 반복 -> cuda 용량 모자람

In [ ]:
# model load
model.load_state_dict(torch.load('./checkpoint/bert_model.pt', map_location=device))

<All keys matched successfully>

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        # print("out=", out, 'label= ', label)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            torch.save(model.state_dict(), './checkpoint/bert_model.pt') # model save
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


In [ ]:
# model save
# torch.save(model.state_dict(), './checkpoint/bert_model.pt')

In [ ]:
# model load
model.load_state_dict(torch.load('./checkpoint/bert_model.pt', map_location=device)) # data 450개
# 데이터 450개 짜리가 데이터 1000개짜리보다 성능이 좋다 싶었는데, 실제로 450이 성능이 더 좋은 것 같음

<All keys matched successfully>

In [ ]:
# 기쁨:0 분노:1 두려움:2 슬픔:3 unknown:4

def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["기쁨", "분노", "두려움", "슬픔", "unknown"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("일기의 감정:", cate[idx])
    print("신뢰도:", "{:.2f}%".format(softmax(result,idx)))

In [ ]:
input_text= "[오늘 하루]  1. 외모에 자신감이 없는 아이가 있다. 다른 친구와 말다툼을 하다가 그 친구가 외모 이야기를 하면 (돼지, 뚱뚱해) 속상해하며 사과받고 싶다고 한다. 이야기 들어주고 사과시킨다. 전체를 대상으로 외모 이야기 하지 말자고 다시 지도한다. 아까 놀림받았던 학생이 이번에는 다른 아이를 놀렸다고 한다. 불러서 이야기해본다. '입장 바꿔서 생각해 보자. 네가 그런 말 들었을때 기분이 어때? ... 네가 그렇게 말해서 00이 마음이 어땠을까? ...앞으로 어떻게 해야 할 것 같아?' 2. 수업 중 게임 하기가 힘들다. 어떻게 팀을 나눠도, 아무리 시작하기 전에 서로 기분 상할 일 없게 하자, 지적하지 말자, 내가 알아서 할 거고 너희는 심판이 아니니까 반칙하는 걸 봐도 이야기하지 마라, 이기고 지는 게 중요하지 않하다가 갑자기 '그런데 지난주에 00이가~' 하면서 다른 이야기를 끌어오거나, 주어 서술어 목적어를 지나치게 생략하거나... 해서 이제는 진술서 양식을 만들어서 거기에 써 오라고 하고 있다. 이렇게 하면 아이들이 쓰는 게 귀찮아서라도 고자질이 줄어든다고 하더라. 그래서 그런가? 조금 줄어든 것도 같다. 하루에 10번 오던 게 3번 정도로 줄어든 느낌이다. 4. 진술서를 써오고 사과받고 싶다고 하는 경우에는 일단 양 쪽 이야기를 모두 들어본다. 그런데 A는 자기가 그런 적이 없다고 하고, B는 A가 분명히 했다고 한다. 내 앞에서 자기들끼리 싸우기 시작한다. 몇 번 겪고 지쳐서 다음부터는 증인을 데려오라고 했더니 그 자리에 있지도 않았던 자기 친구를 데려와서 증인이랍시고 내세운다. 이런 건 어디서 배운 걸까?"

In [ ]:
testModel(model, input_text)

일기의 감정: 슬픔
신뢰도: 42.54%
